In [1]:
import pandas as pd 
import numpy as np
import anndata as an
import pytorch_lightning as pl
import torch

import sys
sys.path.append('../src')

from models.lib.data import *
from models.lib.neural import *
from models.lib.lightning_train import *

In [2]:
df = an.read_h5ad('../data/cortex_development/matrix_T.h5ad')
labels = pd.read_csv('../data/cortex_development/labels.tsv', sep='\t')
labels

,Cell,WGCNAcluster,Name,Age_in_Weeks,RegionName,Laminae,Area
0,Hi_GW21_4.Hi_GW21_4,Glyc,Sample1,19.00,Cortex,All,0
1,Hi_GW21_5.Hi_GW21_5,tRG,Sample1,19.00,Cortex,All,0
2,Hi_GW21_8.Hi_GW21_8,RG-div1,Sample1,19.00,Cortex,All,0
3,Hi_GW21_1.Hi_GW21_1,nEN-early2,Sample1,19.00,Cortex,All,0
4,Hi_GW21_2.Hi_GW21_2,nEN-early2,Sample1,19.00,Cortex,All,0
...,...,...,...,...,...,...,...
4256,S222.G8,RG-early,Sample48,5.85,Presumptive_cortex,All,0
4257,S222.G9,U4,Sample48,5.85,Presumptive_cortex,All,0
4258,S222.H4,RG-early,Sample48,5.85,Presumptive_cortex,All,0
4259,S222.H7,RG-early,Sample48,5.85,Presumptive_cortex,All,0


In [3]:
labels = labels.dropna(subset=['WGCNAcluster'])
labels.index.name = 'cell'
labels.to_csv('../data/cortex_development/labels_clean.tsv', sep='\t')

In [4]:
labels

,Cell,WGCNAcluster,Name,Age_in_Weeks,RegionName,Laminae,Area
cell,,,,,,,
0,Hi_GW21_4.Hi_GW21_4,Glyc,Sample1,19.00,Cortex,All,0
1,Hi_GW21_5.Hi_GW21_5,tRG,Sample1,19.00,Cortex,All,0
2,Hi_GW21_8.Hi_GW21_8,RG-div1,Sample1,19.00,Cortex,All,0
3,Hi_GW21_1.Hi_GW21_1,nEN-early2,Sample1,19.00,Cortex,All,0
4,Hi_GW21_2.Hi_GW21_2,nEN-early2,Sample1,19.00,Cortex,All,0
...,...,...,...,...,...,...,...
4256,S222.G8,RG-early,Sample48,5.85,Presumptive_cortex,All,0
4257,S222.G9,U4,Sample48,5.85,Presumptive_cortex,All,0
4258,S222.H4,RG-early,Sample48,5.85,Presumptive_cortex,All,0


In [10]:
module = DataModule(
    datafiles=['../data/cortex_development/matrix_T.h5ad'],
    labelfiles=['../data/cortex_development/labels_clean.tsv'],
    sep='\t',
    class_label='WGCNAcluster',
    index_col='cell',
    is_assumed_numeric=False,
    batch_size=32,
    normalize=True,
)

In [11]:
module.prepare_data()
module.setup()

is_assumed_numeric=False, using sklearn.preprocessing.LabelEncoder and encoding target variables.
Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


In [12]:
data = an.read_h5ad('../data/cortex_development/matrix_T.h5ad')
data.shape

(4261, 56864)

In [13]:
len(module.trainloader.dataset) + len(module.valloader.dataset) + len(module.testloader.dataset)

4129

In [14]:
next(iter(module.trainloader))

(tensor([[0.0000, 0.0000, 0.0071,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0077,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0082,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0074,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0056,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0097,  ..., 0.0000, 0.0000, 0.0000]]),
 tensor([ 0, 12, 37, 17, 38, 38, 16, 12, 37, 37, 35, 41, 20,  4, 40, 37,  6, 38,
         37, 38, 41, 44,  6, 19, 30, 10, 13, 23, 13, 37, 37,  3]))

In [30]:
labels = pd.read_csv('../data/cortex_development/labels_clean.tsv', sep='\t')
labels.to_csv('../data/cortex_development/labels_clean.csv', index=False)


In [33]:
module = DataModule(
    datafiles=['../data/cortex_development/matrix_T.h5ad'],
    labelfiles=['../data/cortex_development/labels_clean.csv'],
    class_label='WGCNAcluster',
    index_col='cell',
    is_assumed_numeric=False,
    batch_size=32,
)

module.setup()

Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


In [36]:
next(iter(module.trainloader))

torch.Size([32])

In [8]:
model = TabNetLightning(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.02,
    },
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.ReduceLROnPlateau,
        'factor': 0.0001,
    }
)

Initializing network
Initializing explain matrix


In [9]:
wandb_logger = WandbLogger(
    project=f"tabnet-classifer-sweep",
    name='Local Cortex Development Model'
)

trainer = pl.Trainer(logger=wandb_logger)

trainer.fit(model, datamodule=module)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(
wandb: Currently logged in as: jlehrer1 (use `wandb login --relogin` to force relogin)



  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 3.8 M 
-----------------------------------
3.8 M     Trainable params
0         Non-trainable params
3.8 M     Total params
15.036    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:111: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/base-data-science/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:685: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
